### 러닝 플랫폼 데이터 추출 사용방법

#### 1.패키지 설치 : posco-lp

#### 2.사용가능한 기능 : lp.login()으로 관리자 화면 로그인에 성공하면 아래 메소드 사용가능

- lp.getCompanyList() → LP에 등록된 그룹사/협력사/해외법인의 정보를 추출해 DataFrame으로 리턴
- lp.getUserList() → LP에서 특정 사용자(교육생)를 검색한 결과를 DataFrame으로 리턴
- lp.getChaList() → 차수 목록을 DataFrame으로 리턴
- lp.getChaDetail() → 과정ID, 차수ID를 입력하면 해당 차수의 상세 정보를 json으로 리턴
- lp.getStudentRegisterList() → 과정ID, 차수ID를 입력하면 해당 차수에 등록된 교육생 정보를 DataFrame으로 리턴
- lp.getStudentPassedList() → 과정ID, 차수ID를 입력하면 해당 차수의 수료자 정보를 DataFrame으로 리턴
- lp.getSurveyMeanScore() → 과정ID, 차수ID를 입력하면 해당 차수의 설문지 개수, 응답자수, 설문별 평균값을 출력
- lp.getLectureRoomInfo() → 입력한 날짜(yyyymmdd)에 해당하는 강의실 예약정보를 DataFrame으로 리턴
- lp.updateStudentToPassed() → 과정ID, 차수ID를 입력하면 해당 차수의 수료조건에 해당되는 교육생들을 일괄 수료처리
- lp.resetUserPassword() → 사용자의 LP관리ID(ex : 986669Y)를 입력하면 해당 사용자의 LP 비밀번호를 초기화

#### 3. 사용하기

- LP 로그인

In [ ]:
from poscolp import LP
import pyautogui as pg

lp_id = "user_id@poscohrd.com"
lp_pw = pg.password(title="lp패스워드", text="패스워드 입력하기")
 
lp = LP(lp_id, lp_pw)
lp.login()

- 차수 리스트 추출

In [ ]:
lp.getChaList()

- 기간별 차수리스트 추출

In [ ]:
lp.getChaList(start_date="20220101", end_date="20221231", simple=True)

- 담당자별 차수 리스트 추출

In [ ]:
ChaList = lp.getChaList(start_date="20220101", end_date="20221231", simple=True)

In [ ]:
ChaList[ChaList['담당자'] == '박민경']

In [ ]:
ChaList[(ChaList['담당자'] == '박민경') | (ChaList['담당자'] == '유구봉')]

- 엑셀로 내보내기

In [ ]:
ChaList = ChaList[(ChaList['담당자'] == '박민경') | (ChaList['담당자'] == '유구봉')]

In [ ]:
ChaList.to_excel('ChaList.xlsx', index=False)

#### 4.응용하기 - 조건(기간/담당자)별 교육생 정보 추출

- 기간별 차수 리스트 추출

In [ ]:
ChaList = lp.getChaList(start_date="20221001", end_date="20221031", simple=True)
ChaList

- 담담자별 차수 리스트 추출

In [ ]:
ChaList2 = ChaList[ChaList['담당자'] == '박민경']
ChaList2

- 과정ID, 차수ID 추출

In [ ]:
id = ChaList2[['과정ID','차수ID']]
id

In [ ]:
id_1 = id.iloc[0,0]
id_1

In [ ]:
id_2 = id.iloc[0,1]
id_2

- ID별 교육생 명단 추출

In [ ]:
lp.getStudentRegisterList(courseId=id_1, chaId=id_2)

- 조건별 교육생 명단 추출

In [ ]:
import pandas as pd

student_list = pd.DataFrame()
for idx in range(len(id)):
      id_1 = id.iloc[idx,0]
      id_2 = id.iloc[idx,1]
      student_list = student_list.append(lp.getStudentRegisterList(courseId=id_1, chaId=id_2))

In [ ]:
student_list

- 수강취소 제거

In [ ]:
student_list2 = student_list[student_list['상태'] != '수강취소']
student_list2

In [ ]:
student_list2.to_excel('sample.xlsx', index=False)

- 코드 종합

In [ ]:
import pandas as pd

ChaList = lp.getChaList(start_date="20221001", end_date="20221031", simple=True)
ChaList2 = ChaList[ChaList['담당자'] == '박민경']
id = ChaList2[['과정ID','차수ID']]

student_list = pd.DataFrame()
for idx in range(len(id)):
      id_1 = id.iloc[idx,0]
      id_2 = id.iloc[idx,1]
      student_list = student_list.append(lp.getStudentRegisterList(courseId=id_1, chaId=id_2))

student_list2 = student_list[student_list['상태'] != '수강취소']
student_list2